# Preprocessing

Source: http://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

In [1]:
import os
import numpy as np
import pandas as pd
import config as cfg

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from pandas_profiling import ProfileReport

pd.set_option("display.max_columns", None)

### Train/test split

In [2]:
df = pd.read_excel(os.path.join("Data", "data_original", "default of credit card clients.xls"), header=1)

df["BAD"] = df["default payment next month"]
df['SEX'] = df['SEX'] - 1
df = df.drop(["ID", "default payment next month"], axis=1)

df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BAD
0,20000,1,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,120000,1,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,1,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,1,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,0,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,0,3,1,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,0,3,2,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,0,2,2,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,0,3,1,41,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [3]:
print("Bad rate:", df["BAD"].mean())

Bad rate: 0.2212


In [4]:
X = df.drop(['BAD'], axis=1)
y = df['BAD']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=cfg.TEST_SIZE, random_state=cfg.SEED, stratify=y)

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

rus = RandomUnderSampler(sampling_strategy=cfg.SAMPLING_STRATEGY)
X_train, y_train = rus.fit_resample(X_train, y_train)

X_train.to_csv(os.path.join("Data", "data_preprocessed", "X_train.csv"), index=False)
X_test.to_csv(os.path.join("Data", "data_preprocessed", "X_test.csv"), index=False)
y_train.to_csv(os.path.join("Data", "data_preprocessed", "y_train.csv"), index=False)
y_test.to_csv(os.path.join("Data", "data_preprocessed", "y_test.csv"), index=False)

ProfileReport(X_train, minimal=True).to_file(os.path.join("Results", "X_train.html"))

Summarize dataset:   0%|          | 0/31 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("Bad rate:", y_train.mean())

X_train: (10507, 23)
X_test: (7500, 23)
Bad rate: 0.47368421052631576


### Train/test split with binning

In [6]:
df_binned = df.copy()

df_binned['LIMIT_BAL'] = pd.qcut(df['LIMIT_BAL'], 10, labels=range(1, 11))
df_binned['AGE'] = pd.qcut(df_binned['AGE'], 10, labels=range(1, 11))
df_binned['BILL_AMT1'] = pd.qcut(df_binned['BILL_AMT1'], 10, labels=range(1, 11))
df_binned['BILL_AMT2'] = pd.qcut(df_binned['BILL_AMT2'], 10, labels=range(1, 11))
df_binned['BILL_AMT3'] = pd.qcut(df_binned['BILL_AMT3'], 10, labels=range(1, 11))
df_binned['BILL_AMT4'] = pd.qcut(df_binned['BILL_AMT4'], 10, labels=range(1, 11))
df_binned['BILL_AMT5'] = pd.qcut(df_binned['BILL_AMT5'], 10, labels=range(1, 11))
df_binned['BILL_AMT6'] = pd.qcut(df_binned['BILL_AMT6'], 10, labels=range(1, 11))
df_binned['PAY_AMT1'] = pd.qcut(df_binned['PAY_AMT1'], 5, labels=range(1, 6))
df_binned['PAY_AMT2'] = pd.qcut(df_binned['PAY_AMT2'], 5, labels=range(1, 6))
df_binned['PAY_AMT3'] = pd.qcut(df_binned['PAY_AMT3'], 5, labels=range(1, 6))
df_binned['PAY_AMT4'] = pd.qcut(df_binned['PAY_AMT4'], 4, labels=range(1, 5))
df_binned['PAY_AMT5'] = pd.qcut(df_binned['PAY_AMT5'], 4, labels=range(1, 5))
df_binned['PAY_AMT6'] = pd.qcut(df_binned['PAY_AMT6'], 4, labels=range(1, 5))
        
df_binned

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BAD
0,1,1,2,1,1,2,2,-1,-1,-2,-2,3,3,2,1,1,1,1,2,1,1,1,1,1
1,5,1,2,2,2,-1,2,0,0,0,2,3,3,3,3,3,4,1,2,2,2,1,3,1
2,4,1,2,2,5,0,0,0,0,0,0,6,5,5,5,5,5,2,2,2,2,2,4,0
3,2,1,2,1,6,0,0,0,0,0,0,7,7,8,6,7,7,3,3,2,2,2,2,0
4,2,0,2,1,10,-1,0,-1,0,0,0,4,4,7,6,6,6,3,5,5,4,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,8,0,3,1,7,0,0,0,0,0,0,10,10,10,9,7,5,5,5,4,3,4,2,0
29996,6,0,3,2,8,-1,-1,-1,-1,0,0,2,3,3,4,4,1,3,4,5,1,1,1,0
29997,1,0,2,2,6,4,3,2,-1,0,0,3,3,3,6,6,6,1,1,5,4,3,3,1
29998,4,0,3,1,8,1,-1,0,0,0,-1,1,8,8,8,5,8,5,4,2,3,4,3,1


In [7]:
print("Bad rate:", df_binned["BAD"].mean())

Bad rate: 0.2212


In [8]:
X = df_binned.drop(['BAD'], axis=1)
y = df_binned['BAD']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=cfg.TEST_SIZE, random_state=cfg.SEED, stratify=y)

rus = RandomUnderSampler(sampling_strategy=cfg.SAMPLING_STRATEGY)
X_train, y_train = rus.fit_resample(X_train, y_train)

X_train.to_csv(os.path.join("Data", "data_preprocessed_binned", "X_train.csv"), index=False)
X_test.to_csv(os.path.join("Data", "data_preprocessed_binned", "X_test.csv"), index=False)
y_train.to_csv(os.path.join("Data", "data_preprocessed_binned", "y_train.csv"), index=False)
y_test.to_csv(os.path.join("Data", "data_preprocessed_binned", "y_test.csv"), index=False)

In [9]:
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("Bad rate:", y_train.mean())

X_train: (10507, 23)
X_test: (7500, 23)
Bad rate: 0.47368421052631576
